# DSGVO-konforme Anonymisierung mit k-Anonymität, l-Diversität und t-Closeness
Dieses Notebook zeigt, wie man einen Datensatz DSGVO-konform anonymisiert:
- Entfernen direkter Identifikatoren
- Generalisierung von Quasi-Identifikatoren
- Prüfung auf k-Anonymität, l-Diversität und t-Closeness


# Code

In [1]:
!pip install pandas scipy

### Import necessary modules

In [11]:
import pandas as pd
from scipy.spatial.distance import euclidean

### Receive Dataset and prepare it


Der Datensatz stellt eine vereinfachte Retail-Datenbasis dar, in der Kundendaten wie Postleitzahl, Alter und Einkommen gespeichert sind. Diese Attribute werden genutzt, um Anonymisierungstechniken wie k-Anonymität, l-Diversität und t-Closeness zu demonstrieren. Dabei werden die Quasi-Identifikatoren (PLZ, Alter) so verallgemeinert, dass einzelne Personen nicht mehr eindeutig identifiziert werden können. Gleichzeitig bleibt die Vielfalt in sensiblen Attributen wie dem Einkommen gewährleistet, damit keine Rückschlüsse auf Einzelne möglich sind. Der Datensatz dient somit als Beispiel, um Methoden des datenschutzgerechten Umgangs mit personenbezogenen Daten praxisnah zu testen und zu evaluieren.

### Link zum Datensatz

https://www.kaggle.com/datasets/bhavikjikadara/retail-transactional-dataset

In [2]:
! pip install -q kaggle
! mkdir ~/.kaggle

Übergabe von Kaggle Benutzer Daten:
{"username":"....","key":"...."}

In [3]:
# ... Hier ihre JSON Cred als dictionary eingeben
d_json_cred ={"username":"lizzldizzl","key":"7126d6d48a18986c8a8704fbb94e4a44"}

Kaggle Zugangsdaten speichern

In [4]:
import pandas as pd
kaggle_cred = pd.DataFrame(d_json_cred, index=[0]).to_json("~/.kaggle/kaggle.json")

Authorisierung geben dass Kaggle Daten heruntergeladen werden dürfen

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d bhavikjikadara/retail-transactional-dataset

Dataset URL: https://www.kaggle.com/datasets/bhavikjikadara/retail-transactional-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/24.8M [00:00<?, ?B/s]
100% 24.8M/24.8M [00:00<00:00, 807MB/s]


Unzip der Daten

In [7]:
!unzip retail-transactional-dataset.zip -d ./data

Archive:  retail-transactional-dataset.zip
  inflating: ./data/retail_data.csv  


### Originaldaten laden

In [9]:
CSV_PATH = "./data/retail_data.csv"
df = pd.read_csv(CSV_PATH)

print("=== Originaldaten ===")
print(df.head(), "\n")

=== Originaldaten ===
   Transaction_ID  Customer_ID                 Name                Email  \
0       8691788.0      37249.0  Michelle Harrington    Ebony39@gmail.com   
1       2174773.0      69749.0          Kelsey Hill     Mark36@gmail.com   
2       6679610.0      30192.0         Scott Jensen    Shane85@gmail.com   
3       7232460.0      62101.0        Joseph Miller     Mary34@gmail.com   
4       4983775.0      27901.0        Debra Coleman  Charles30@gmail.com   

          Phone                      Address        City            State  \
0  1.414787e+09            3959 Amanda Burgs    Dortmund           Berlin   
1  6.852900e+09           82072 Dawn Centers  Nottingham          England   
2  8.362160e+09            4133 Young Canyon     Geelong  New South Wales   
3  2.776752e+09  8148 Thomas Creek Suite 100    Edmonton          Ontario   
4  9.098268e+09    5813 Lori Ports Suite 269     Bristol          England   

   Zipcode    Country  ...  Total_Amount Product_Category 

### Originaldaten filtern

In [10]:
# Nur die gewünschten Spalten behalten
keep_cols = ["Zipcode", "Age", "Country", "Income"]
df_reduced = df[keep_cols]

# Neue CSV speichern
OUTPUT_PATH = "./data/retail_data_filtered.csv"
df_reduced.to_csv(OUTPUT_PATH, index=False)

print("Neue CSV gespeichert:", OUTPUT_PATH)
print(df_reduced.head())


Neue CSV gespeichert: ./data/retail_data_filtered.csv
   Zipcode   Age    Country Income
0  77985.0  21.0    Germany    Low
1  99071.0  19.0         UK    Low
2  75929.0  48.0  Australia    Low
3  88420.0  56.0     Canada   High
4  48704.0  22.0         UK    Low


### Anonymität

Hier wird zunächst eine gefilterte CSV-Datei eingelesen, die nur die Spalten Zipcode, Age, Country und Income enthält. Anschließend werden die Quasi-Identifikatoren generalisiert, indem Postleitzahlen auf die ersten zwei Ziffern reduziert und Alterswerte in Intervalle (≤30, 31-50, 51+) eingeteilt werden. Danach wird die k-Anonymität überprüft, sodass nur Gruppen mit mindestens zwei Datensätzen erhalten bleiben. Im nächsten Schritt erfolgt die l-Diversitätsprüfung, bei der sichergestellt wird, dass jede Gruppe mindestens zwei verschiedene Ausprägungen des sensiblen Attributes „Income“ aufweist. Schließlich wird die t-Closeness berechnet, um zu prüfen, ob die Verteilung von „Income“ innerhalb der Gruppen der Gesamtverteilung ähnelt, bevor die anonymisierten Daten ausgegeben werden.

In [12]:
import pandas as pd
from scipy.spatial.distance import euclidean

# 1. Gefilterte CSV laden (nur Zipcode, Age, Country, Income enthalten)
CSV_PATH = "./data/retail_data_filtered.csv"
df = pd.read_csv(CSV_PATH)

# 2. Quasi-Identifikatoren generalisieren
# - PLZ auf die ersten 2 Stellen reduzieren
# - Alter in Intervalle gruppieren
df['Zipcode'] = df['Zipcode'].astype(str).str[:2]
df['Age'] = pd.cut(df['Age'], bins=[0, 30, 50, 100],
                   labels=['<=30', '31-50', '51+'])

# 3. k-Anonymität prüfen
k = 2
quasi_identifiers = ['Zipcode', 'Age']  # Kombination der Quasi-Identifikatoren
# Gruppengröße berechnen
group_sizes = df.groupby(quasi_identifiers).size().reset_index(name='Count')
# Nur Gruppen behalten, die mindestens k Datensätze haben
df_k_anonym = df.merge(group_sizes[group_sizes['Count'] >= k],
                       on=quasi_identifiers)

print(f"Nach k-Anonymität (k={k}): {len(df_k_anonym)} von {len(df)} Datensätzen\n")

# 4. l-Diversität prüfen
l = 2
sensitive_attr = 'Income'  # sensibles Attribut

# Prüffunktion: eine Gruppe ist l-divers, wenn sie mind. l verschiedene Werte enthält
def check_l_diversity(group):
    return group[sensitive_attr].nunique() >= l

# Nur Gruppen behalten, die l-divers sind
df_l_diverse = df_k_anonym.groupby(quasi_identifiers).filter(check_l_diversity)
print(f"Nach l-Diversität (l={l}): {len(df_l_diverse)} von {len(df_k_anonym)} Datensätzen\n")

# 5. t-Closeness prüfen
t = 0.3
# Gesamtverteilung des sensiblen Attributs
overall_dist = df[sensitive_attr].value_counts(normalize=True)

# Prüffunktion: eine Gruppe ist t-close, wenn ihre Verteilung nahe genug an der Gesamtverteilung liegt
def t_closeness(group):
    group_dist = group[sensitive_attr].value_counts(normalize=True)
    # gleiche Reihenfolge wie in overall_dist erzwingen, fehlende Werte = 0
    group_dist = group_dist.reindex(overall_dist.index, fill_value=0)
    return euclidean(overall_dist.values, group_dist.values) <= t

# Nur Gruppen behalten, die t-close sind
df_t_close = df_l_diverse.groupby(quasi_identifiers).filter(t_closeness)
print(f"Nach t-Closeness (t={t}): {len(df_t_close)} von {len(df_l_diverse)} Datensätzen\n")

# 6. Ergebnis anzeigen
print("=== Anonymisierte Daten (ersten 10 Zeilen) ===")
print(df_t_close.head(10))


=== Originaldaten (gefiltert) ===
   Zipcode   Age    Country Income
0  77985.0  21.0    Germany    Low
1  99071.0  19.0         UK    Low
2  75929.0  48.0  Australia    Low
3  88420.0  56.0     Canada   High
4  48704.0  22.0         UK    Low 



/tmp/ipython-input-928675150.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_sizes = df.groupby(quasi_identifiers).size().reset_index(name='Count')
/tmp/ipython-input-928675150.py:38: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_l_diverse = df_k_anonym.groupby(quasi_identifiers).filter(check_l_diversity)


Nach k-Anonymität (k=2): 301837 von 302010 Datensätzen

Nach l-Diversität (l=2): 301837 von 301837 Datensätzen



/tmp/ipython-input-928675150.py:54: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_t_close = df_l_diverse.groupby(quasi_identifiers).filter(t_closeness)


Nach t-Closeness (t=0.3): 301837 von 301837 Datensätzen

=== Anonymisierte Daten (ersten 10 Zeilen) ===
  Zipcode    Age    Country  Income  Count
0      77   <=30    Germany     Low   1423
1      99   <=30         UK     Low   1968
2      75  31-50  Australia     Low    972
3      88    51+     Canada    High    561
4      48   <=30         UK     Low   1668
5      74    51+  Australia  Medium    485
6      47   <=30     Canada     Low   1660
7      86   <=30    Germany  Medium   1521
8      39  31-50  Australia  Medium   1180
9      64   <=30    Germany  Medium   1512


# Interpretation

„Nach Anwendung der Generalisierung und Entfernung direkter Identifikatoren wurde der Datensatz so reduziert, dass er die Anforderungen an k-Anonymität (k=2), l-Diversität (l=2) und t-Closeness (t=0.3) vollständig erfüllt. 301 837 Datensätze verblieben anonymisiert, was 99,94 % des Originaldatensatzes entspricht.“

Ergebnisse der Anonymisierung
Der Datensatz wurde durch Generalisierung der Postleitzahlen sowie die Bildung von Altersgruppen vereinfacht. Anschließend erfolgte die schrittweise Prüfung der Datenschutzkriterien k-Anonymität, l-Diversität und t-Closeness. Mit k=2 konnte sichergestellt werden, dass jede Kombination der Quasi-Identifikatoren mindestens zwei Datensätze umfasst. Die l-Diversität (l=2) gewährleistete eine ausreichende Vielfalt beim sensiblen Attribut Income, und die t-Closeness (t=0.3) bestätigte, dass die Einkommensverteilungen in den Gruppen der Gesamtverteilung sehr nahekommen. Insgesamt erfüllten 301 837 von 302 010 Einträgen die Kriterien, was einer Abdeckung von 99,94 % entspricht und die hohe Datenqualität trotz Anonymisierung verdeutlicht.

Fazit
Die Ergebnisse zeigen, dass durch zielgerichtete Generalisierung und Diversitätsprüfungen ein sehr hoher Anteil des Datensatzes anonymisiert werden konnte, ohne dabei wesentliche Informationen zu verlieren. Damit wird deutlich, dass die gewählten Verfahren eine effektive Grundlage für die datenschutzkonforme Weiterverarbeitung von Retail-Daten bieten.